## **Helper Functions**
We have several helper functions to help us in our testing and parsing:
- `bin_to_twos_complement` takes a binary string and outputs the 2's complement integer representation, necessary in this case since python simply denotes negative binary strings as `'-0b....'`.
- `parse` takes a value `i` which may be in hexadecimal, or Lucid shorthand, and converts to a binary representation.
- `dec_to_tcbin` takes an integer value and converts it to its 2's complement representation binary string. Since negative values are incorrectly represented by Python, we first invert all the bits and add one, before converting to a binary string in order to get the correct representation.
- `alufn` takes our `a`, `b`, and `fn` values and outputs the corresponding value expected from the operation. 

In [2]:
def bin_to_twos_complement(num):
    return -2**(len(num) - 1) * int(num[0]) + int(num[1:], 2)

def parse(i):
    if i[0] == "h":
        return bin(int(i[1:], 16))[2:].zfill(16)
    elif i[0] == "b":
        return i[1:]
    elif i[0:2] == "16":
        return bin(int(i[3:]))[2:].zfill(16)
    elif i[0] == "6":
        return bin(int(i[2:]))[2:].zfill(6)

def dec_to_tcbin(x):
    if x < 0:
        x = (abs(x) ^ 4294967295) + 1
    return bin(x)[2:].zfill(32)[16:]

def alufn(bin_a, bin_b, fn):
    a = int(bin_a, 2)
    b = int(bin_b, 2)
    fn = int(fn, 2)

    if fn == 0:
        return dec_to_tcbin(a + b)
    elif fn == 1:
        return dec_to_tcbin(a - b)
    elif fn == 4:
        return dec_to_tcbin(a * b)
    elif fn == 16:
        return dec_to_tcbin(a ^ b)
    elif fn == 17:
        return dec_to_tcbin(a & b)
    elif fn == 18:
        return dec_to_tcbin(a)
    elif fn == 19:
        return dec_to_tcbin(a | b)
    elif fn == 32:
        return dec_to_tcbin(a << b)
    elif fn == 33:
        return dec_to_tcbin(a >> b)
    elif fn == 34:
        msb = bin(a)[2:].zfill(16)[0]
        return (msb * (16 - len(bin(a >> b)[2:])) + bin(a >> b)[2:])
    elif fn == 35:
        return (bin(a)[2:].zfill(16)[b:] + bin(a)[2:].zfill(16)[0 : b])
    elif fn == 36:
        return (bin(a)[2:].zfill(16)[16 - b:] + bin(a)[2:].zfill(16)[0 : 16 - b])
    elif fn == 51:
        return "0" * 15 + str(int(bin_to_twos_complement(bin_a) == bin_to_twos_complement(bin_b)))
    elif fn == 53:
        return "0" * 15 + str(int(bin_to_twos_complement(bin_a) < bin_to_twos_complement(bin_b)))
    elif fn == 55:
        return "0" * 15 + str(int(bin_to_twos_complement(bin_a) <= bin_to_twos_complement(bin_b)))
    else:
        raise Exception("Invalid ALUFN code")

## **Checking Basic Operations**
The cell below was used for small-scale testing of ALU operations, and whether they were sufficient to cover the edge case we were testing.

In [8]:
a = "0000"
b = "0001"

# Find binary representation of a and b from their hex representation
a_bin = bin(int(a, 16))[2:].zfill(16)
b_bin = bin(int(b, 16))[2:].zfill(16)

# Find their 2's complement decimal value
a_val = bin_to_twos_complement(a_bin)
b_val = bin_to_twos_complement(b_bin)

# Perform operation and get truncated binary value
truncated_result = dec_to_tcbin(a_val - b_val)

# Print operation in binary
print("Binary Operands and Output:")
print(f"  {a_bin}")
print(f"- {b_bin}")
print("------------------")
print(f"  {truncated_result}\n")

# Print operation in decimal
print("Decimal Operands and Output:")
print(f"{a_val:>18}")
print(f"- {b_val:>16}")
print("-" * 18)
print(f"{bin_to_twos_complement(truncated_result):>18}\n")

Binary Operands and Output:
  0000000000000000
- 0000000000000001
------------------
  1111111111111111

Decimal Operands and Output:
                 0
-                1
------------------
                -1



## **Bitshift Testing**
The code below was used to test our bitshift offsets prior to using the `alufn` function to batch-generate testcases.

In [15]:
c = "DEAD"

# Get integer representation of c
c_num = int(c, 16)

# Get 2's complement binary representation of c
c_bin = dec_to_tcbin(c_num)

# Get MSB of c to use for SRA
msb = c_bin[0]

# Indicate if we want to perform SRA (True) or SHR (False)
signed = True

print(f"Offset:  0 -> {c}")
# Test non-byte boundary, byte boundary, and max offset for each operation
for i in [5, 8, 15]:
    # # Test SHL
    out = hex(int(dec_to_tcbin(c_num << i), 2))
    # # Test SHR
    # out = hex(int(dec_to_tcbin(c_num >> i), 2))
    # # Test SRA
    # out = (msb * (16 - len(bin(c_num >> i)[2:])) + bin(c_num >> i)[2:]
    print(f"Offset: {i:>2} -> {out[2:].upper()}")

Offset:  0 -> DEAD
Offset:  5 -> D5A0
Offset:  8 -> AD00
Offset: 15 -> 8000


## **Troubleshooting**
`cases.txt` contained our original handwritten test cases, which contained bugs. In the subsequent cells, we debug as well as parse the test cases to prepare our FSM.

### **FSM**
To create the states and their outputs for our FSM, we took our test cases, and only kept the input portion, as that would end up being the output of the FSM. We formatted the data such that each row has an input corresponding to its row number, so that could just copy the data straight into Lucid.

In [19]:
# Read the original file
with open("cases.txt") as fp:
    arr = fp.read().split("\n")

# 'out' stores the output to be copy-pasted into the ROM
out = []
# 'fsm_out' stores the output to be copy-pasted into the FSM
fsm_out = []
# We copy a dictionary 'temp' for every test-case row
temp = {"a": 0, "b":0, "alufn": 0, "out": 0, "z": 0, "v": 0, "n": 0}
# We init a counter to keep track of how many actual rows are testcases, so that
# we can enumerate our testcases for the FSM, while keeping comments for the ROM file
# to keep things clear
count = 1

# Enumerate through each row in the original file
for idx, val in enumerate(arr):
    # If the row is a comment or an empty line, just append to the ROM file and continue
    if val.strip() == "" or val.strip()[0:2] == "//":
        out.append(val)
        continue
    # Else split the row on ';'
    else:
        val = val.split(";")

    # Index 0 will be our data, index 1 (if exists) will be that row's comments
    comment = val[1]
    # Split on ': '
    val = val[0].split(": ")
    # x contains our address input to the ROM / address output of the FSM
    x = val[0][2:-1].split(", ")
    # y contains our output from the ROM
    y = val[1][8:-1].split(", ")

    # Load and parse the values from that row into our dictionary
    foo = temp.copy()
    foo["a"] = parse(x[0])
    foo["b"] = parse(x[1])
    foo["alufn"] = parse(x[2])
    foo["out"] = parse(y[0])
    foo["z"] = y[1]
    foo["v"] = y[2]
    foo["n"] = y[3]

    # Calculate what the correct output should be for the specified operands
    result = alufn(foo['a'], foo['b'], foo['alufn'])

    # Calculate z, v, n values for our operands
    inv_b = bin(int(foo["b"], 2) ^ int(foo["alufn"][-1] * 16, 2))[2:].zfill(16)
    zvn_result = alufn(foo['a'], foo["b"], foo['alufn'][-1])
    z = int(not any([int(i) for i in list(zvn_result)]))
    v = int(
        (int(foo["a"][0], 2) and int(inv_b[0], 2) and not int(zvn_result[0], 2)) or
        (not int(foo["a"][0], 2) and not int(inv_b[0], 2) and int(zvn_result[0], 2))
    )
    n = int(zvn_result[0])

    # Append our formatted results to their respective lists
    out.append(f"b{foo['a']}{foo['b']}{foo['alufn']}: out = b{result}{z}{v}{n};{comment}")
    fsm_out.append(f"{str(int(count - 1))}: out = b{foo['a']}{foo['b']}{foo['alufn']};")

    # DEBUGGING
    # Print and increment our row count 
    print(count)    
    count += 1

    # Print a, b, and alufn
    print(f"{foo['a']} | {foo['b']} | {foo['alufn']}")
    # Print the expected output, and the output we originally calculated from our handcrafted test cases
    print(f"{result} | {foo['out']}")

    # Print the expected z, v, n values, and the z, v, n values we originally calculated from our handcrafted test cases
    print(f"{z}, {v}, {n} | {foo['z']}, {foo['v']}, {foo['n']}")
    print()

# Write our ROM data to 'parsed_rom.txt'
with open("parsed_rom.txt", "w+") as fp:
    fp.write("\n".join(out))

# Write our FSM data to 'parsed_fsm.txt'
with open("parsed_fsm.txt", "w+") as fp:
    fp.write("\n".join(fsm_out))


1
0000000000000000 | 0000000000000000 | 000000
0000000000000000 | 0000000000000000
1, 0, 0 | 1, 0, 0

2
0111111111111111 | 0000000000000000 | 000000
0111111111111111 | 0111111111111111
0, 0, 0 | 0, 0, 1

3
0000000000000000 | 0111111111111111 | 000000
0111111111111111 | 0111111111111111
0, 0, 0 | 0, 0, 1

4
1111111111111111 | 0000000000000000 | 000000
1111111111111111 | 1111111111111111
0, 0, 1 | 0, 0, 1

5
0000000000000000 | 1111111111111111 | 000000
1111111111111111 | 1111111111111111
0, 0, 1 | 0, 0, 1

6
0001001000110100 | 0011010001010110 | 000000
0100011010001010 | 0100011010001010
0, 0, 0 | 0, 0, 0

7
1111100000000001 | 1111100000100000 | 000000
1111000000100001 | 1111000000100001
0, 0, 1 | 0, 0, 1

8
1101111000000000 | 0000000010101101 | 000000
1101111010101101 | 1101111010101101
0, 0, 1 | 0, 0, 1

9
0000000011101111 | 1011111000000000 | 000000
1011111011101111 | 1011111011101111
0, 0, 1 | 0, 0, 1

10
1111111111111111 | 1111111111111111 | 000000
1111111111111110 | 111111111111111